In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
from scipy.spatial import ConvexHull

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir3='/Volumes/My Passport/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
areas_merra2=ds['cell_area']/1e6

lats_array_merra2=xr.zeros_like(areas_merra2)
for i in range(lats_array_merra2.lon.size):
    lats_array_merra2[:,i]=lats_array_merra2.lat.values

ds=xr.open_dataset('/Users/ahenny/Downloads/jra55_gridareas_125.nc')
areas_jra55=ds['cell_area']/1e6

lats_array_jra55=xr.zeros_like(areas_jra55)
for i in range(lats_array_jra55.lon.size):
    lats_array_jra55[:,i]=lats_array_jra55.lat.values

lats_list_era5tp=[]
lats_list_era5reid=[]
lats_list_era5mk=[]
lats_list_merra2lo=[]
lats_list_merra2cndl=[]
lats_list_merra2ac=[]
lats_list_jra55gw=[]

yrs=np.arange(1980,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    #binary=ar
    binary=binary.where(binary==1)
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_era5tp.append(lat_mean)
    print(lat_mean)
    
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    #binary=ar
    binary=binary.where(binary==1)
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_era5reid.append(lat_mean)
    print(lat_mean)
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    ar=ds['ar']
    land_mask_interp=land_mask.interp_like(ar[0,:,:])
    binary=ar.where(land_mask_interp>=0.5)
    #binary=ar
    binary=binary.where(binary==1)
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_era5mk.append(lat_mean)
    print(lat_mean)
    
    ###
    
    if 1==0:
        ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
        ar=ds['ar']
        #land_mask_interp=land_mask.interp_like(ar[0,:,:])
        #binary=ar.where(land_mask_interp>=0.5)
        binary=ar
        binary=binary.where(binary==1)

        lats_array_merra2['lat']=binary.lat.values
        lats_array_merra2['lon']=binary.lon.values
        lats_all=xr.zeros_like(binary)
        for j in range(binary.time.size):
            lats_all[j,:,:]=lats_array_merra2

        lat_sel=lats_all.where(binary==1)
        lat_sel=np.absolute(lat_sel)
        weights=np.cos(np.deg2rad(lat_sel.lat))
        lat_sel=lat_sel.weighted(weights)
        lat_mean=lat_sel.mean().values.tolist()
        lats_list_merra2lo.append(lat_mean)
        print(lat_mean)

        ###

        ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
        ar=ds['ar']
        #land_mask_interp=land_mask.interp_like(ar[0,:,:])
        #binary=ar.where(land_mask_interp>=0.5)
        binary=ar
        binary=binary.where(binary==1)

        lats_array_merra2['lat']=binary.lat.values
        lats_array_merra2['lon']=binary.lon.values
        lats_all=xr.zeros_like(binary)
        for j in range(binary.time.size):
            lats_all[j,:,:]=lats_array_merra2

        lat_sel=lats_all.where(binary==1)
        lat_sel=np.absolute(lat_sel)
        weights=np.cos(np.deg2rad(lat_sel.lat))
        lat_sel=lat_sel.weighted(weights)
        lat_mean=lat_sel.mean().values.tolist()
        lats_list_merra2cndl.append(lat_mean)
        print(lat_mean)

        ###

        ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
        ar=ds['ar']
        #land_mask_interp=land_mask.interp_like(ar[0,:,:])
        #binary=ar.where(land_mask_interp>=0.5)
        binary=ar
        binary=binary.where(binary==1)

        lats_array_merra2['lat']=binary.lat.values
        lats_array_merra2['lon']=binary.lon.values
        lats_all=xr.zeros_like(binary)
        for j in range(binary.time.size):
            lats_all[j,:,:]=lats_array_merra2

        lat_sel=lats_all.where(binary==1)
        lat_sel=np.absolute(lat_sel)
        weights=np.cos(np.deg2rad(lat_sel.lat))
        lat_sel=lat_sel.weighted(weights)
        lat_mean=lat_sel.mean().values.tolist()
        lats_list_merra2ac.append(lat_mean)
        print(lat_mean)

        ###

        ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
        ar=ds['ar']
        #land_mask_interp=land_mask.interp_like(ar[0,:,:])
        #binary=ar.where(land_mask_interp>=0.5)
        binary=ar
        binary=binary.where(binary==1)
        binary['lon']=areas_jra55.lon.values
        binary['lat']=areas_jra55.lat.values

        lats_array_jra55['lat']=binary.lat.values
        lats_array_jra55['lon']=binary.lon.values
        lats_all=xr.zeros_like(binary)
        for j in range(binary.time.size):
            lats_all[j,:,:]=lats_array_jra55

        lat_sel=lats_all.where(binary==1)
        lat_sel=np.absolute(lat_sel)
        weights=np.cos(np.deg2rad(lat_sel.lat))
        lat_sel=lat_sel.weighted(weights)
        lat_mean=lat_sel.mean().values.tolist()
        lats_list_jra55gw.append(lat_mean)
        print(lat_mean)

In [ ]:
dk=xr.Dataset()
dk['lats_list_era5mk']=(('time'),lats_list_era5mk)
dk['lats_list_era5reid']=(('time'),lats_list_era5reid)
dk['lats_list_era5tp']=(('time'),lats_list_era5tp)

dk.coords['time']=np.arange(1980,2020,1)

try:
    os.remove(dir_data+'land_lat_intercomparison_era5artmip_CORRECTED.nc')#land
except OSError:
    pass
dk.to_netcdf(dir_data+'land_lat_intercomparison_era5artmip_CORRECTED.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas_merra2=ds['cell_area']/1e6

lats_list_era5_variable=[]
lats_list_era5_fixed=[]
lats_list_era5_fixed_poleward=[]
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_era5_variable.append(lat_mean)
    print(lat_mean)
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_era5_fixed.append(lat_mean)
    print(lat_mean)
    
    ###
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_era5_fixed_poleward.append(lat_mean)
    print(lat_mean)

In [ ]:
dk=xr.Dataset()
dk['lats_list_era5_variable']=(('time'),lats_list_era5_variable)
dk['lats_list_era5_fixed']=(('time'),lats_list_era5_fixed)
dk['lats_list_era5_fixed_poleward']=(('time'),lats_list_era5_fixed_poleward)

dk.coords['time']=np.arange(1940,2024,1)

try:
    os.remove(dir_data+'global_lat_intercomparison_era5_CORRECTED.nc')#land
except OSError:
    pass
dk.to_netcdf(dir_data+'global_lat_intercomparison_era5_CORRECTED.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir2+'merra2_gridareas.nc')
areas_merra2=ds['cell_area']/1e6

lats_list_merra2_variable=[]
lats_list_merra2_fixed=[]
lats_list_merra2_fixed_poleward=[]
yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_merra2_variable.append(lat_mean)
    print(lat_mean)
    
    ###
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_merra2_fixed.append(lat_mean)
    print(lat_mean)
    
    ###
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_merra2['lat']=binary.lat.values
    lats_array_merra2['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_merra2

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_merra2_fixed_poleward.append(lat_mean)
    print(lat_mean)

In [ ]:
ds=xr.open_dataset(dir_data+'jra55_gridareas.nc')
areas_jra55=ds['cell_area']/1e6

lats_array_jra55=xr.zeros_like(areas_jra55)
for i in range(lats_array_jra55.lon.size):
    lats_array_jra55[:,i]=lats_array_jra55.lat.values

lats_list_jra55_variable=[]
lats_list_jra55_fixed=[]
lats_list_jra55_fixed_poleward=[]
yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
    t2m=ds['t2m'][0,:,:]
    land_mask=t2m/t2m
    land_mask=land_mask.fillna(0)
        
    lon_east=[x for x in land_mask.longitude.values if x>=180.]
    lon_west=[x for x in land_mask.longitude.values if x<180.]
    l_east=land_mask.sel(longitude=lon_east)
    l_west=land_mask.sel(longitude=lon_west)
    l_east['longitude']=[x-360. for x in lon_east]
    land_mask=xr.concat([l_east,l_west],dim='longitude')
        
    dk=xr.Dataset()
    dk['land']=(('lat','lon'),land_mask.values)
    dk.coords['lat']=land_mask.latitude.values
    dk.coords['lon']=land_mask.longitude.values
        
    land_mask=dk['land']
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_jra55['lat']=binary.lat.values
    lats_array_jra55['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_jra55

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_jra55_variable.append(lat_mean)
    print(lat_mean)
    
    ###
    
    ds=xr.open_dataset(dir_data+'jra55.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_jra55['lat']=binary.lat.values
    lats_array_jra55['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_jra55

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_jra55_fixed.append(lat_mean)
    print(lat_mean)
    
    ###
    
    ds=xr.open_dataset(dir2+'jra55.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    land_mask_interp=land_mask.interp_like(labeled[0,:,:])
    #labeled=labeled.where(land_mask_interp>=0.5)
    binary=labeled/labeled
    
    lats_array_jra55['lat']=binary.lat.values
    lats_array_jra55['lon']=binary.lon.values
    lats_all=xr.zeros_like(binary)
    for j in range(binary.time.size):
        lats_all[j,:,:]=lats_array_jra55

    lat_sel=lats_all.where(binary==1)
    lat_sel=np.absolute(lat_sel)
    weights=np.cos(np.deg2rad(lat_sel.lat))
    lat_sel=lat_sel.weighted(weights)
    lat_mean=lat_sel.mean().values.tolist()
    lats_list_jra55_fixed_poleward.append(lat_mean)
    print(lat_mean)

In [ ]:
dk=xr.Dataset()
dk['lats_list_era5_variable']=(('time'),lats_list_era5_variable)
dk['lats_list_era5_fixed']=(('time'),lats_list_era5_fixed)
dk['lats_list_era5_fixed_poleward']=(('time'),lats_list_era5_fixed_poleward)

dk['lats_list_merra2_variable']=(('time1'),lats_list_merra2_variable)
dk['lats_list_merra2_fixed']=(('time1'),lats_list_merra2_fixed)
dk['lats_list_merra2_fixed_poleward']=(('time1'),lats_list_merra2_fixed_poleward)

dk['lats_list_jra55_variable']=(('time2'),lats_list_jra55_variable)
dk['lats_list_jra55_fixed']=(('time2'),lats_list_jra55_fixed)
dk['lats_list_jra55_fixed_poleward']=(('time2'),lats_list_jra55_fixed_poleward)

dk['lats_list_era5tp']=(('time3'),lats_list_era5tp)
dk['lats_list_era5reid']=(('time3'),lats_list_era5reid)
dk['lats_list_era5mk']=(('time3'),lats_list_era5mk)

dk['lats_list_merra2lo']=(('time3'),lats_list_merra2lo)
dk['lats_list_merra2cndl']=(('time3'),lats_list_merra2cndl)
dk['lats_list_merra2ac']=(('time3'),lats_list_merra2ac)

dk['lats_list_jra55gw']=(('time3'),lats_list_jra55gw)

dk.coords['time']=np.arange(1940,2024,1)
dk.coords['time1']=np.arange(1980,2024,1)
dk.coords['time2']=np.arange(1958,2024,1)
dk.coords['time3']=np.arange(1980,2020,1)

try:
    os.remove(dir_data+'global_lat_intercomparison.nc')#land
except OSError:
    pass
dk.to_netcdf(dir_data+'global_lat_intercomparison.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'global_lat_intercomparison.nc')

lats_list_merra2_variable=ds['lats_list_merra2_variable'].values.tolist()
lats_list_merra2_fixed=ds['lats_list_merra2_fixed'].values.tolist()
lats_list_merra2_fixed_poleward=ds['lats_list_merra2_fixed_poleward'].values.tolist()

lats_list_jra55_variable=ds['lats_list_jra55_variable'].values.tolist()
lats_list_jra55_fixed=ds['lats_list_jra55_fixed'].values.tolist()
lats_list_jra55_fixed_poleward=ds['lats_list_jra55_fixed_poleward'].values.tolist()

#lats_list_era5tp=ds['lats_list_era5tp'].values.tolist()
#lats_list_era5reid=ds['lats_list_era5reid'].values.tolist()
#lats_list_era5mk=ds['lats_list_era5mk'].values.tolist()

lats_list_merra2lo=ds['lats_list_merra2lo'].values.tolist()
lats_list_merra2cndl=ds['lats_list_merra2cndl'].values.tolist()
lats_list_merra2ac=ds['lats_list_merra2ac'].values.tolist()

lats_list_jra55gw=ds['lats_list_jra55gw'].values.tolist()

ds=xr.open_dataset(dir_data+'global_lat_intercomparison_era5_CORRECTED.nc')
lats_list_era5_variable=ds['lats_list_era5_variable'].values.tolist()
lats_list_era5_fixed=ds['lats_list_era5_fixed'].values.tolist()
lats_list_era5_fixed_poleward=ds['lats_list_era5_fixed_poleward'].values.tolist()


ds=xr.open_dataset(dir_data+'global_lat_intercomparison_era5artmip_CORRECTED.nc')
lats_list_era5tp=ds['lats_list_era5tp'].values.tolist()
lats_list_era5reid=ds['lats_list_era5reid'].values.tolist()
lats_list_era5mk=ds['lats_list_era5mk'].values.tolist()

In [ ]:
sigs_list=[]
#mean=float(sum(areas_list_era5_variable))/float(len(areas_list_era5_variable))
#new_era5_variable=[x/mean for x in areas_list_era5_variable]
#mean=float(sum(areas_list_era5_fixed))/float(len(areas_list_era5_fixed))
#new_era5_fixed=[x/mean for x in areas_list_era5_fixed]
#mean=float(sum(areas_list_era5_fixed_poleward))/float(len(areas_list_era5_fixed_poleward))
#new_era5_fixed_poleward=[x/mean for x in areas_list_era5_fixed_poleward]

#mean=float(sum(areas_list_merra2_variable))/float(len(areas_list_merra2_variable))
#new_merra2_variable=[x/mean for x in areas_list_merra2_variable]
#mean=float(sum(areas_list_merra2_fixed))/float(len(areas_list_merra2_fixed))
#new_merra2_fixed=[x/mean for x in areas_list_merra2_fixed]
#mean=float(sum(areas_list_merra2_fixed_poleward))/float(len(areas_list_merra2_fixed_poleward))
#new_merra2_fixed_poleward=[x/mean for x in areas_list_merra2_fixed_poleward]

#mean=float(sum(areas_list_jra55_variable))/float(len(areas_list_jra55_variable))
#new_jra55_variable=[x/mean for x in areas_list_jra55_variable]
#mean=float(sum(areas_list_jra55_fixed))/float(len(areas_list_jra55_fixed))
#new_jra55_fixed=[x/mean for x in areas_list_jra55_fixed]
#mean=float(sum(areas_list_jra55_fixed_poleward))/float(len(areas_list_jra55_fixed_poleward))
#new_jra55_fixed_poleward=[x/mean for x in areas_list_jra55_fixed_poleward]

new_era5_variable=lats_list_era5_variable
new_era5_fixed=lats_list_era5_fixed
new_era5_fixed_poleward=lats_list_era5_fixed_poleward

new_merra2_variable=lats_list_merra2_variable
new_merra2_fixed=lats_list_merra2_fixed
new_merra2_fixed_poleward=lats_list_merra2_fixed_poleward

new_jra55_variable=lats_list_jra55_variable
new_jra55_fixed=lats_list_jra55_fixed
new_jra55_fixed_poleward=lats_list_jra55_fixed_poleward

list_all=lats_list_era5_variable[40:]+lats_list_era5_fixed[40:]+lats_list_era5_fixed_poleward[40:]+lats_list_jra55_variable[22:]+lats_list_jra55_fixed[22:]+lats_list_jra55_fixed_poleward[22:]+lats_list_merra2_variable+lats_list_merra2_fixed+lats_list_merra2_fixed_poleward
print(len(list_all))
mean_all=float(sum(list_all))/float(len(list_all))
mean_all=mean_all

test_array=np.zeros((9,84))
testa=[np.nan]*40+new_merra2_fixed
testb=[np.nan]*40+new_merra2_fixed_poleward
testc=[np.nan]*40+new_merra2_variable

testa1=[np.nan]*18+new_jra55_fixed
testb1=[np.nan]*18+new_jra55_fixed_poleward
testc1=[np.nan]*18+new_jra55_variable

testa2=new_era5_fixed
testb2=new_era5_fixed_poleward
testc2=new_era5_variable

test_array[0,:]=testa
test_array[1,:]=testb
test_array[2,:]=testc
test_array[3,:]=testa1
test_array[4,:]=testb1
test_array[5,:]=testc1
test_array[6,:]=testa2
test_array[7,:]=testb2
test_array[8,:]=testc2

avg_value=np.nanmean(test_array,axis=0)
min_value=np.nanmin(test_array,axis=0)
max_value=np.nanmax(test_array,axis=0)


#mean=float(sum(areas_list_era5tp))/float(len(areas_list_era5tp))
#new_era5tp=[x/mean for x in areas_list_era5tp]
#mean=float(sum(areas_list_era5reid))/float(len(areas_list_era5reid))
#new_era5reid=[x/mean for x in areas_list_era5reid]
#mean=float(sum(areas_list_era5mk))/float(len(areas_list_era5mk))
#new_era5mk=[x/mean for x in areas_list_era5mk]

#mean=float(sum(areas_list_merra2lo))/float(len(areas_list_merra2lo))
#new_merra2lo=[x/mean for x in areas_list_merra2lo]
#mean=float(sum(areas_list_merra2cndl))/float(len(areas_list_merra2cndl))
#new_merra2cndl=[x/mean for x in areas_list_merra2cndl]
#mean=float(sum(areas_list_merra2ac))/float(len(areas_list_merra2ac))
#new_merra2ac=[x/mean for x in areas_list_merra2ac]

#mean=float(sum(areas_list_jra55gw))/float(len(areas_list_jra55gw))
#new_jra55gw=[x/mean for x in areas_list_jra55gw]
    
new_era5tp=lats_list_era5tp
new_era5reid=lats_list_era5reid
new_era5mk=lats_list_era5mk

new_merra2lo=lats_list_merra2lo
new_merra2cndl=lats_list_merra2cndl
new_merra2ac=lats_list_merra2ac

new_jra55gw=lats_list_jra55gw
    
yrs=np.arange(1980,2024,1)
count=0
test=new_era5_variable[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed_poleward[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1
    

test=new_jra55_variable[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed_poleward[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1


test=new_merra2_variable
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed_poleward
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=avg_value
r=st.linregress(yrs,avg_value[40:])
change=r[0]*len(yrs)

yrs=np.arange(1980,2020,1)
r=st.linregress(yrs,new_era5tp)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_era5tp))/float(len(lats_list_era5tp))
mean=mean
label_1='TempestLR ERA5 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5reid)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_era5reid))/float(len(lats_list_era5reid))
mean=mean
label_2='Reid500 ERA5 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5mk)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_era5mk))/float(len(lats_list_era5mk))
mean=mean
label_3='Mundhenk_v3 ERA5 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2lo)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_merra2lo))/float(len(lats_list_merra2lo))
mean=mean
label_4='Lora_v2 MERRA-2 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2cndl)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_merra2cndl))/float(len(lats_list_merra2cndl))
mean=mean
label_5='ClimateNet_DL MERRA-2 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2ac)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_merra2ac))/float(len(lats_list_merra2ac))
mean=mean
label_6='ARCONNECT_v2 MERRA-2 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_jra55gw)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_jra55gw))/float(len(lats_list_jra55gw))
mean=mean
label_7='GuanWaliser JRA-55 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'


label='Variants (Mean='+str(round(mean_all,2))+', '+'$\Delta$='+str(round(change,2))+', '+str(count)+' significant)'
    
#label='AR land area, '+str(count)+' significant'
yrs=np.arange(1940,2024,1)
fig=plt.figure(figsize=(8,16))
#fig=plt.figure(figsize=(14,8))
ax=plt.subplot(1,1,1)


width=1.5
ax.plot(np.arange(1940,2024,1),lats_list_era5_variable,color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1940,2024,1),lats_list_era5_fixed,color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1940,2024,1),lats_list_era5_fixed_poleward,color='k',linewidth=width)

ax.plot(np.arange(1980,2024,1),lats_list_merra2_variable,color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1980,2024,1),lats_list_merra2_fixed,color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1980,2024,1),lats_list_merra2_fixed_poleward,color='k',linewidth=width)

ax.plot(np.arange(1958,2024,1),lats_list_jra55_variable,color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1958,2024,1),lats_list_jra55_fixed,color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1958,2024,1),lats_list_jra55_fixed_poleward,color='k',linewidth=width)


ax.fill_between([1939,1979.5],25.25,54,color='gray',alpha=0.4)


ax.fill_between(yrs,min_value,max_value,color='k',alpha=0.3)

#plt.axhline(y=1,color='k',linewidth=1)
ax.plot(np.arange(1940,2024,1),avg_value,color='k',linewidth=8,label=label,zorder=100)

yrs=np.arange(1980,2020,1)
width=6

ax.plot(yrs,new_merra2lo,color='b',linewidth=width,label=label_4)
ax.plot(yrs,new_merra2cndl,color='cyan',linewidth=width,label=label_5)
ax.plot(yrs,new_merra2ac,color='mediumorchid',linewidth=width,label=label_6)

ax.plot(yrs,new_era5tp,color='limegreen',linewidth=width,label=label_1)
ax.plot(yrs,new_era5reid,color='saddlebrown',linewidth=width,label=label_2)
ax.plot(yrs,new_era5mk,color='orange',linewidth=width,label=label_3)

ax.plot(yrs,new_jra55gw,color='deeppink',linewidth=width,label=label_7)



ax.set_ylim(25.75,52)
ax.set_xlim(1939,2024)
ax.tick_params(labelsize=17)
ax.set_xlabel('Year',fontsize=20)
ax.set_ylabel('Degrees poleward',fontsize=20)
ax.grid(True,color='gray')
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{global}$'+' '+r'$\bf{latitude}$',fontsize=32,pad=5)
#plt.legend(loc='upper center',fontsize=15,ncol=1)
plt.legend(loc='upper center',fontsize=14,ncol=1)
plt.tight_layout()
#plt.yticks([0.7,0.8,0.9,1,1.1,1.2,1.3,1.4],labels=['-30%','-20%','-10%','0','+10%','+20%','+30%','+40%'])
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(2)

plt.show()

In [ ]:
fig.savefig(dir_data+'lat_land_redo_1a.png',dpi=200)

In [ ]:
ds=xr.open_dataset(dir_data+'land_lat_intercomparison.nc')

lats_list_era5_variable=ds['lats_list_era5_variable'].values.tolist()
lats_list_era5_fixed=ds['lats_list_era5_fixed'].values.tolist()
lats_list_era5_fixed_poleward=ds['lats_list_era5_fixed_poleward'].values.tolist()

lats_list_merra2_variable=ds['lats_list_merra2_variable'].values.tolist()
lats_list_merra2_fixed=ds['lats_list_merra2_fixed'].values.tolist()
lats_list_merra2_fixed_poleward=ds['lats_list_merra2_fixed_poleward'].values.tolist()

lats_list_jra55_variable=ds['lats_list_jra55_variable'].values.tolist()
lats_list_jra55_fixed=ds['lats_list_jra55_fixed'].values.tolist()
lats_list_jra55_fixed_poleward=ds['lats_list_jra55_fixed_poleward'].values.tolist()

#lats_list_era5tp=ds['lats_list_era5tp'].values.tolist()
#lats_list_era5reid=ds['lats_list_era5reid'].values.tolist()
#lats_list_era5mk=ds['lats_list_era5mk'].values.tolist()

lats_list_merra2lo=ds['lats_list_merra2lo'].values.tolist()
lats_list_merra2cndl=ds['lats_list_merra2cndl'].values.tolist()
lats_list_merra2ac=ds['lats_list_merra2ac'].values.tolist()

lats_list_jra55gw=ds['lats_list_jra55gw'].values.tolist()

ds=xr.open_dataset(dir_data+'land_lat_intercomparison_era5artmip_CORRECTED.nc')
lats_list_era5tp=ds['lats_list_era5tp'].values.tolist()
lats_list_era5reid=ds['lats_list_era5reid'].values.tolist()
lats_list_era5mk=ds['lats_list_era5mk'].values.tolist()

In [ ]:
#mean=float(sum(areas_list_era5_variable))/float(len(areas_list_era5_variable))
#new_era5_variable=[x/mean for x in areas_list_era5_variable]
#mean=float(sum(areas_list_era5_fixed))/float(len(areas_list_era5_fixed))
#new_era5_fixed=[x/mean for x in areas_list_era5_fixed]
#mean=float(sum(areas_list_era5_fixed_poleward))/float(len(areas_list_era5_fixed_poleward))
#new_era5_fixed_poleward=[x/mean for x in areas_list_era5_fixed_poleward]

#mean=float(sum(areas_list_merra2_variable))/float(len(areas_list_merra2_variable))
#new_merra2_variable=[x/mean for x in areas_list_merra2_variable]
#mean=float(sum(areas_list_merra2_fixed))/float(len(areas_list_merra2_fixed))
#new_merra2_fixed=[x/mean for x in areas_list_merra2_fixed]
#mean=float(sum(areas_list_merra2_fixed_poleward))/float(len(areas_list_merra2_fixed_poleward))
#new_merra2_fixed_poleward=[x/mean for x in areas_list_merra2_fixed_poleward]

#mean=float(sum(areas_list_jra55_variable))/float(len(areas_list_jra55_variable))
#new_jra55_variable=[x/mean for x in areas_list_jra55_variable]
#mean=float(sum(areas_list_jra55_fixed))/float(len(areas_list_jra55_fixed))
#new_jra55_fixed=[x/mean for x in areas_list_jra55_fixed]
#mean=float(sum(areas_list_jra55_fixed_poleward))/float(len(areas_list_jra55_fixed_poleward))
#new_jra55_fixed_poleward=[x/mean for x in areas_list_jra55_fixed_poleward]

new_era5_variable=lats_list_era5_variable
new_era5_fixed=lats_list_era5_fixed
new_era5_fixed_poleward=lats_list_era5_fixed_poleward

new_merra2_variable=lats_list_merra2_variable
new_merra2_fixed=lats_list_merra2_fixed
new_merra2_fixed_poleward=lats_list_merra2_fixed_poleward

new_jra55_variable=lats_list_jra55_variable
new_jra55_fixed=lats_list_jra55_fixed
new_jra55_fixed_poleward=lats_list_jra55_fixed_poleward

list_all=lats_list_era5_variable[40:]+lats_list_era5_fixed[40:]+lats_list_era5_fixed_poleward[40:]+lats_list_jra55_variable[22:]+lats_list_jra55_fixed[22:]+lats_list_jra55_fixed_poleward[22:]+lats_list_merra2_variable+lats_list_merra2_fixed+lats_list_merra2_fixed_poleward
print(len(list_all))
mean_all=float(sum(list_all))/float(len(list_all))
mean_all=mean_all

test_array=np.zeros((9,84))
testa=[np.nan]*40+new_merra2_fixed
testb=[np.nan]*40+new_merra2_fixed_poleward
testc=[np.nan]*40+new_merra2_variable

testa1=[np.nan]*18+new_jra55_fixed
testb1=[np.nan]*18+new_jra55_fixed_poleward
testc1=[np.nan]*18+new_jra55_variable

testa2=new_era5_fixed
testb2=new_era5_fixed_poleward
testc2=new_era5_variable

test_array[0,:]=testa
test_array[1,:]=testb
test_array[2,:]=testc
test_array[3,:]=testa1
test_array[4,:]=testb1
test_array[5,:]=testc1
test_array[6,:]=testa2
test_array[7,:]=testb2
test_array[8,:]=testc2

avg_value=np.nanmean(test_array,axis=0)
min_value=np.nanmin(test_array,axis=0)
max_value=np.nanmax(test_array,axis=0)


#mean=float(sum(areas_list_era5tp))/float(len(areas_list_era5tp))
#new_era5tp=[x/mean for x in areas_list_era5tp]
#mean=float(sum(areas_list_era5reid))/float(len(areas_list_era5reid))
#new_era5reid=[x/mean for x in areas_list_era5reid]
#mean=float(sum(areas_list_era5mk))/float(len(areas_list_era5mk))
#new_era5mk=[x/mean for x in areas_list_era5mk]

#mean=float(sum(areas_list_merra2lo))/float(len(areas_list_merra2lo))
#new_merra2lo=[x/mean for x in areas_list_merra2lo]
#mean=float(sum(areas_list_merra2cndl))/float(len(areas_list_merra2cndl))
#new_merra2cndl=[x/mean for x in areas_list_merra2cndl]
#mean=float(sum(areas_list_merra2ac))/float(len(areas_list_merra2ac))
#new_merra2ac=[x/mean for x in areas_list_merra2ac]

#mean=float(sum(areas_list_jra55gw))/float(len(areas_list_jra55gw))
#new_jra55gw=[x/mean for x in areas_list_jra55gw]
    
new_era5tp=lats_list_era5tp
new_era5reid=lats_list_era5reid
new_era5mk=lats_list_era5mk

new_merra2lo=lats_list_merra2lo
new_merra2cndl=lats_list_merra2cndl
new_merra2ac=lats_list_merra2ac

new_jra55gw=lats_list_jra55gw
    
yrs=np.arange(1980,2024,1)
count=0
test=new_era5_variable[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_era5_fixed_poleward[40:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1
    

test=new_jra55_variable[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_jra55_fixed_poleward[22:]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1


test=new_merra2_variable
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=new_merra2_fixed_poleward
r=st.linregress(yrs,test)
sigs_list.append(r[3])
if r[3]<=0.03551:
    count=count+1

test=avg_value
r=st.linregress(yrs,avg_value[40:])
change=r[0]*len(yrs)

yrs=np.arange(1980,2020,1)
r=st.linregress(yrs,new_era5tp)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_era5tp))/float(len(lats_list_era5tp))
mean=mean
label_1='TempestLR ERA5 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5reid)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_era5reid))/float(len(lats_list_era5reid))
mean=mean
label_2='Reid500 ERA5 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_era5mk)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_era5mk))/float(len(lats_list_era5mk))
mean=mean
label_3='Mundhenk_v3 ERA5 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2lo)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_merra2lo))/float(len(lats_list_merra2lo))
mean=mean
label_4='Lora_v2 MERRA-2 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2cndl)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_merra2cndl))/float(len(lats_list_merra2cndl))
mean=mean
label_5='ClimateNet_DL MERRA-2 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_merra2ac)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_merra2ac))/float(len(lats_list_merra2ac))
mean=mean
label_6='ARCONNECT_v2 MERRA-2 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'

r=st.linregress(yrs,new_jra55gw)
sigs_list.append(r[3])
change1=r[0]*len(yrs)
mean=float(sum(lats_list_jra55gw))/float(len(lats_list_jra55gw))
mean=mean
label_7='GuanWaliser JRA-55 (Mean='+str(round(mean,2))+', '+'$\Delta$='+str(round(change1,2))+', p='+str(round(r[3],3))+')'


label='Variants (Mean='+str(round(mean_all,2))+', '+'$\Delta$='+str(round(change,2))+', '+str(count)+' significant)'
    
#label='AR land area, '+str(count)+' significant'
yrs=np.arange(1940,2024,1)
#fig=plt.figure(figsize=(11,14))
fig=plt.figure(figsize=(8,16))
ax=plt.subplot(1,1,1)


width=1.5
ax.plot(np.arange(1940,2024,1),lats_list_era5_variable,color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1940,2024,1),lats_list_era5_fixed,color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1940,2024,1),lats_list_era5_fixed_poleward,color='k',linewidth=width)

ax.plot(np.arange(1980,2024,1),lats_list_merra2_variable,color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1980,2024,1),lats_list_merra2_fixed,color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1980,2024,1),lats_list_merra2_fixed_poleward,color='k',linewidth=width)

ax.plot(np.arange(1958,2024,1),lats_list_jra55_variable,color='k',linewidth=width,linestyle=':')
ax.plot(np.arange(1958,2024,1),lats_list_jra55_fixed,color='k',linewidth=width,linestyle='--')
ax.plot(np.arange(1958,2024,1),lats_list_jra55_fixed_poleward,color='k',linewidth=width)


ax.fill_between([1939,1979.5],25.25,54,color='gray',alpha=0.4)


ax.fill_between(yrs,min_value,max_value,color='k',alpha=0.3)

#plt.axhline(y=1,color='k',linewidth=1)
ax.plot(np.arange(1940,2024,1),avg_value,color='k',linewidth=8,label=label,zorder=100)

yrs=np.arange(1980,2020,1)
width=6

ax.plot(yrs,new_merra2lo,color='b',linewidth=width,label=label_4)
ax.plot(yrs,new_merra2cndl,color='cyan',linewidth=width,label=label_5)
ax.plot(yrs,new_merra2ac,color='mediumorchid',linewidth=width,label=label_6)

ax.plot(yrs,new_era5tp,color='limegreen',linewidth=width,label=label_1)
ax.plot(yrs,new_era5reid,color='saddlebrown',linewidth=width,label=label_2)
ax.plot(yrs,new_era5mk,color='orange',linewidth=width,label=label_3)

ax.plot(yrs,new_jra55gw,color='deeppink',linewidth=width,label=label_7)



ax.set_ylim(25.75,52)
ax.set_xlim(1939,2024)
ax.tick_params(labelsize=17)
ax.set_xlabel('Year',fontsize=20)
ax.set_ylabel('Degrees poleward',fontsize=20)
ax.grid(True,color='gray')
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{land}$'+' '+r'$\bf{latitude}$',fontsize=32,pad=5)
#plt.legend(loc='upper center',fontsize=15,ncol=1)
plt.legend(loc='upper center',fontsize=14,ncol=1)
plt.tight_layout()
#plt.yticks([0.7,0.8,0.9,1,1.1,1.2,1.3,1.4],labels=['-30%','-20%','-10%','0','+10%','+20%','+30%','+40%'])
for axis in ['bottom','top','left','right']:
    ax.spines[axis].set_linewidth(2)

plt.show()

In [ ]:
fig.savefig(dir_data+'lat_land_redo_2a.png',dpi=200)

In [ ]:
print(sigs_list)